# Transferring Senator Data From ProPublica to SQLite DB

In [1]:
# Config file
import configparser

# Serverless SQL database
import sqlite3

# API and webscrape libraries
import requests
import re
from bs4 import BeautifulSoup

## Webscrape Congress Numbers

In [2]:
# Get ProPublica documentation
url = 'https://projects.propublica.org/api-docs/congress-api/members/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
# Find string with available house and senate
param_string = soup.find('td', string='congress').find_next_sibling().text
print(param_string) # As of December 21st, 2020

102-116 for House, 80-116 for Senate


In [4]:
# Regex the congresses available from API
sen_range = str(re.search(r'(?<=House, ).*(?= for Senate)', param_string)[0])
sen_range = sen_range.split('-')
sen_range = range(int(sen_range[0]), int(sen_range[1])+1)
house_range = str(re.search(r'.*(?= for H)', param_string)[0])
house_range = house_range.split('-')
house_range = range(int(house_range[0]), int(house_range[1])+1)

In [5]:
print(sen_range)
print(house_range)

range(80, 117)
range(102, 117)


## API Data to SQL DB

In [6]:
# Config API
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config.get('propublica', 'PROPUBLICA_API_KEY')

In [7]:
# Instantiate SQLite DB
conn = sqlite3.connect('cong_80_116.db')

In [8]:
# Determine columns for database (Note: 101st Congress and above has 45 columns, 3 are constructed)
r = requests.get(
        f'https://api.propublica.org/congress/v1/80/senate/members.json',
        headers={'X-API-Key': api_key}
)
sample = r.json()['results'][0]['members'][0]
cols = list(sample.keys())
print(f'First 5 columns: {cols[:5]}')
print(f'Number of columns: {len(cols)}')

First 5 columns: ['id', 'title', 'short_title', 'api_uri', 'first_name']
Number of columns: 42


In [9]:
# Add congress number column and format for 
cols = [ col + ' TEXT' for col in cols ]
cols = ', '.join(cols)
cols = cols + ', congress TEXT'

In [10]:
# Create senator table
c = conn.cursor()
c.execute(
    f'''
    CREATE TABLE senators
    ({cols})
    ;
    '''
)
c.close()

In [25]:
# Check columns
c = conn.cursor()
c.execute(
    '''
    PRAGMA table_info(senators)
    ;
    '''
)
print(c.fetchall())
c.close()

[(0, 'id', 'TEXT', 0, None, 0), (1, 'title', 'TEXT', 0, None, 0), (2, 'short_title', 'TEXT', 0, None, 0), (3, 'api_uri', 'TEXT', 0, None, 0), (4, 'first_name', 'TEXT', 0, None, 0), (5, 'middle_name', 'TEXT', 0, None, 0), (6, 'last_name', 'TEXT', 0, None, 0), (7, 'suffix', 'TEXT', 0, None, 0), (8, 'date_of_birth', 'TEXT', 0, None, 0), (9, 'gender', 'TEXT', 0, None, 0), (10, 'party', 'TEXT', 0, None, 0), (11, 'leadership_role', 'TEXT', 0, None, 0), (12, 'twitter_account', 'TEXT', 0, None, 0), (13, 'facebook_account', 'TEXT', 0, None, 0), (14, 'youtube_account', 'TEXT', 0, None, 0), (15, 'govtrack_id', 'TEXT', 0, None, 0), (16, 'cspan_id', 'TEXT', 0, None, 0), (17, 'votesmart_id', 'TEXT', 0, None, 0), (18, 'icpsr_id', 'TEXT', 0, None, 0), (19, 'crp_id', 'TEXT', 0, None, 0), (20, 'google_entity_id', 'TEXT', 0, None, 0), (21, 'fec_candidate_id', 'TEXT', 0, None, 0), (22, 'url', 'TEXT', 0, None, 0), (23, 'rss_url', 'TEXT', 0, None, 0), (24, 'contact_form', 'TEXT', 0, None, 0), (25, 'in_offic

In [12]:
# Function to get data from ProPublica API
def get_congress(n, chamber):
    r = requests.get(
        f'https://api.propublica.org/congress/v1/{n}/{chamber}/members.json',
        headers={'X-API-Key': api_key}
    )
    results = r.json()['results'][0]['members']
    return results

# Function to extract congressional members
def get_members(n, congress):
    if len(congress[0]) == 43:
        members = [ tuple(list(member.values()) + [f'{n}']) for member in congress ]
    else:
        members = [ tuple(list(member.values())[:42] + [f'{n}']) for member in congress ]
    return members

# Function to insert members
def insert_members(n, chamber):
    # SQL insert statement
    s = ','.join('?'*43) # 43 value columns
    insert_statement = f'''INSERT INTO senators VALUES ({s})'''
    
    congress = get_congress(n, chamber)
    members = get_members(n, congress)
    c = conn.cursor()
    c.executemany(insert_statement, members)
    c.close()

In [14]:
# Set parameters
congresses = range(80, 117) # from the 80th congress to the 116th congress

# Insert members
for n in congresses:
    insert_members(n, 'senate')

In [18]:
c = conn.cursor()
c.execute(
    '''
    SELECT * FROM senators
    ;
    '''
)
db = c.fetchall()
c.close()

In [24]:
db[0]

('A000062',
 'Senator, 3rd Class',
 'Sen.',
 'https://api.propublica.org/congress/v1/members/A000062.json',
 'George',
 'David',
 'Aiken',
 None,
 '1892-08-20',
 'M',
 'R',
 None,
 None,
 None,
 None,
 '400722',
 None,
 None,
 '52',
 None,
 None,
 '',
 '',
 None,
 None,
 '0',
 None,
 None,
 None,
 '6',
 None,
 None,
 None,
 '2013-04-12 20:11:44 -0400',
 'ocd-division/country:us/state:vt',
 None,
 None,
 None,
 'VT',
 '3',
 '',
 '',
 '80')